In [1]:
# !pip install -q transformers datasets onnx==1.14.1 onnxruntime optimum[onnxruntime]==1.13.2 accelerate loralib py7zr
# !pip install -q git+https://github.com/huggingface/peft.git

!pip install -q datasets onnx onnxruntime optimum[onnxruntime] py7zr

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.1 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import re
import os
import time
import torch
import onnx
import requests
import onnxruntime as ort
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import Dataset, load_dataset

# Load data

In [3]:
dataset = load_dataset('samsum')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to C:/Users/yevhe/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
test_samples = dataset['test'][:5]

# Measure model runtime

In [5]:
def measure_model_runtime(pipe, data):
    start_time = time.perf_counter()
    for text in data['dialogue']:
        outputs = pipe(text)
    end_time = time.perf_counter()

    return end_time - start_time

# Only Finetuned model

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
tokenizer = AutoTokenizer.from_pretrained('t5-small')

pipe = pipeline('summarization', model=model, tokenizer=tokenizer)

C:\Users\yevhe\PycharmProjects\iasa_nlp_course\venv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yevhe\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
original_model_runtime = measure_model_runtime(pipe, test_samples)
print(f"Original model: {original_model_runtime} sec")

Your max_length is set to 200, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 200, but your input_length is only 155. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=77)
Your max_length is set to 200, but your input_length is only 196. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


Original model: 75.2072215 sec


# ONNX

In [14]:
!optimum-cli export onnx --model t5-small --task summarization ./model/t5-small

Framework not specified. Using pt to export to ONNX.
The task `text2text-generation` was manually specified, and past key values will not be reused in the decoding. if needed, please pass `--task text2text-generation-with-past` to export using the past key values.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
C:\Users\yevhe\PycharmProjects\iasa_nlp_course\venv\lib\site-packages\transformers\modeling_utils.py:873: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:
Post-processing the export

In [9]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from optimum.pipelines import pipeline as onnx_pipeline

tokenizer = AutoTokenizer.from_pretrained("./model/t5-small")
model = ORTModelForSeq2SeqLM.from_pretrained("./model/t5-small", use_cache=False)

In [11]:
onnx_pipe = onnx_pipeline('summarization', model=model, tokenizer=tokenizer)

onnx_model_runtime = measure_model_runtime(pipe, test_samples)
print(f"ONNX model: {onnx_model_runtime} sec")
print(f"ONNX speedup: {round(original_model_runtime / onnx_model_runtime, 2)}x")

Your max_length is set to 200, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 200, but your input_length is only 155. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=77)
Your max_length is set to 200, but your input_length is only 196. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


ONNX model: 14.400055500000008 sec
ONNX speedup: 5.22x


Based on the prediction speed, we can see that the ONNX model much faster than the original model.

# ONNX + Quantization

In [18]:
from onnxruntime.quantization import quantize_dynamic

quantize_dynamic("./model/t5-small/decoder_model.onnx", "./model/t5-small/q_int8/decoder_model.onnx")
quantize_dynamic("./model/t5-small/encoder_model.onnx", "./model/t5-small/q_int8/encoder_model.onnx")

Ignore MatMul due to non constant B: /[/decoder/block.0/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/decoder/block.0/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/decoder/block.0/layer.1/EncDecAttention/MatMul]
Ignore MatMul due to non constant B: /[/decoder/block.0/layer.1/EncDecAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/decoder/block.1/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/decoder/block.1/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/decoder/block.1/layer.1/EncDecAttention/MatMul]
Ignore MatMul due to non constant B: /[/decoder/block.1/layer.1/EncDecAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/decoder/block.2/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/decoder/block.2/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/decoder/block.2/layer.1/EncDecAttention/MatMul]
Ignore MatMul due to non constant 

Ignore MatMul due to non constant B: /[/block.0/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/block.0/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/block.1/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/block.1/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/block.2/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/block.2/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/block.3/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/block.3/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/block.4/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/block.4/layer.0/SelfAttention/MatMul_1]
Ignore MatMul due to non constant B: /[/block.5/layer.0/SelfAttention/MatMul]
Ignore MatMul due to non constant B: /[/block.5/layer.0/SelfAttention/MatMul_1]


In [19]:
tokenizer = AutoTokenizer.from_pretrained("./model/t5-small/q_int8")
model = ORTModelForSeq2SeqLM.from_pretrained("./model/t5-small/q_int8", use_cache=False)

In [21]:
onnx_pipe = onnx_pipeline('summarization', model=model, tokenizer=tokenizer)

onnx_quant_model_runtime = measure_model_runtime(pipe, test_samples)
print(f"Quantized ONNX model: {onnx_quant_model_runtime} sec")
print(f"Quantized ONNX speedup: {round(original_model_runtime / onnx_quant_model_runtime, 2)}x")

Your max_length is set to 200, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 200, but your input_length is only 155. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=77)
Your max_length is set to 200, but your input_length is only 196. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


Quantized ONNX model: 14.28944179999985 sec
Quantized ONNX speedup: 5.26x


On the other hand, the quantizing the ONNX model does not improve the prediction speed in my case, but reduce the model size.

# Model Deployment

In [23]:
test_samples['dialogue'][0]

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [24]:
requests.post(
    'http://127.0.0.1:8000/summarize',
    data=test_samples['dialogue'][0]
    ).text



'{"summary":"Hannah: file_gif> Amanda: Sorry, can\'t find it . he called her last time we were at the park together Hannah ."}'

# Conclusion

Converting to ONNX format helps with CPU inference incredibly, quantizing the model didn't increase performance in my case, but it probably varies from architecture to architecture and model to model.

Note: I freaking hate google colab as I previously tried for 2 days to export a model to ONNX format and failed because of CuPY error and other bugs.